# Stoichiometric model of WL-Pathway

In this notebook, a stoichiometric model of the Wood-Ljungdahl pathway is created, using an existing Genome-Scale Metabolic Model of Clostridium ljungdahlii

In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/PublishedModels/C-ljungdahlii_iHN637.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


To simplify the model, 3 changes are made:

The ATP maintenance reaction is unconstrained, so that a flux is not forced for non-growth associated maintenance

Acetate transport is dependent on a symporter, since only the core metabolism is investigated here, the reaction is replaced by a more simple transporter.

The reaction MTHFD2 uses NADPH as a cofactor, to simplify the model, this cofactor is replaced by NADH. This way there is only one

In [4]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

#Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

#mthfd2 = Reaction("MTHFD2")
#mthfd2.name = "Methylenetetrahydrofolate dehydrogenase (NAD)"
#
#mthfd2.add_metabolites({
#    Cljung_Model.metabolites.mlthf_c: -1.0,
#    Cljung_Model.metabolites.nad_c: -1.0,
#    Cljung_Model.metabolites.methf_c: 1.0,
#    Cljung_Model.metabolites.nadh_c: 1.0,
#})
#mthfd2.bounds = -1000.0, 1000.0
#print(mthfd2.check_mass_balance())    #ensure correct mass balance
#print(mthfd2.bounds)                  #Ensure correct bounds
#print(mthfd2.reaction)

Cljung_Model.add_reactions([acT])#, mthfd2])
#Cljung_Model.add_reactions([mthfd2])


Unconstrain reactions that have been diasbled in the C.ljungdahlii model

In [5]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0

In [6]:
Cljung_Model.reactions.H2td

Reaction identifier,H2td
Name,Hydrogen transport
Memory address,0x07fb844b808d0
Stoichiometry,h2_c <=> h2_e Hydrogen <=> Hydrogen
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport:
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
    
#Carbon fixation to acetyl-CoA
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,

#Product formation from acetyl-CoA
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,

Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

#Exchanges and sinks
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h2_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h2o_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("co2_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("co_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("ac_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("etoh_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("btd_RR_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("but_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("btoh_e"), type="exchange")

WLmodel.add_boundary(WLmodel.metabolites.get_by_id("atp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("adp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadh_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadph_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nad_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("fdxo_42_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("fdxr_42_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("coa_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("pi_c"), type="sink")

WLmodel.reactions

[<Reaction H2td at 0x7fb844b808d0>,
 <Reaction H2Ot at 0x7fb844b6fb70>,
 <Reaction CO2t at 0x7fb844a18390>,
 <Reaction COt at 0x7fb844a580f0>,
 <Reaction ACt at 0x7fb844d9e2b0>,
 <Reaction ETOHt at 0x7fb844b00d30>,
 <Reaction BTDt_RR at 0x7fb844954860>,
 <Reaction BUTt at 0x7fb844995780>,
 <Reaction BTOHt at 0x7fb844954da0>,
 <Reaction FDH7 at 0x7fb844b884e0>,
 <Reaction FTHFLi at 0x7fb844bc4dd8>,
 <Reaction MTHFC at 0x7fb844d32cc0>,
 <Reaction MTHFD at 0x7fb844d3b278>,
 <Reaction MTHFR5 at 0x7fb844d3bc50>,
 <Reaction METR at 0x7fb844c274a8>,
 <Reaction CODH4 at 0x7fb844adc828>,
 <Reaction CODH_ACS at 0x7fb844ae72e8>,
 <Reaction PTAr at 0x7fb844dd8da0>,
 <Reaction ACKr at 0x7fb8449f0c18>,
 <Reaction ACAFDOR at 0x7fb844a83898>,
 <Reaction ALCD2x at 0x7fb845042780>,
 <Reaction POR at 0x7fb844ebdcc0>,
 <Reaction ACLS at 0x7fb8449e8198>,
 <Reaction ACLDC at 0x7fb8449fd278>,
 <Reaction BTDD_RR at 0x7fb844a9a630>,
 <Reaction ACACT1r at 0x7fb84496edd8>,
 <Reaction HACD1 at 0x7fb844b80f60>,
 <

## Add Acetone production pathway

In [9]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [10]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [11]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    #WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [12]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07fb844c96cf8
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fb844d47cc0
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

## Change medium, run FBA, and visualize in Escher

In [15]:
medium = WLmodel.medium

medium['EX_co2_e'] = 10
medium['EX_co_e'] = 0
medium['EX_ac_e'] = 0
medium['EX_etoh_e'] = 0
medium['EX_btd_RR_e'] = 0
medium['EX_but_e'] = 0
medium['EX_btoh_e'] = 0
medium['EX_h2_e'] = 1000

WLmodel.medium = medium

In [16]:
WLmodelActn.reactions.CODH_ACS.bounds = 1,1
WLmodelActn.objective = "EX_ac_e"
solution = WLmodelActn.optimize()
WLmodelActn.optimize()

,fluxes,reduced_costs
H2td,0.0,-0.0
H2Ot,-2.0,-0.0
CO2t,2.0,0.0
COt,0.0,-0.0
ACt,1.0,0.0
...,...,...
SK_pi_c,0.0,-0.0
ctfAB_ac,0.0,0.0
ADC,0.0,0.0
ACTt,0.0,0.0


In [17]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn_SK.json'), reaction_data = solution.fluxes.to_dict())


Builder(reaction_data={'H2td': 0.0, 'H2Ot': -2.0, 'CO2t': 2.0, 'COt': 0.0, 'ACt': 1.0, 'ETOHt': 0.0, 'BTDt_RR'…

In [18]:
cobra.io.write_sbml_model(WLmodelActn, join(data_dir,"Models/WLPathway_sink.xml"))#, use_fbc_package=False)